In [22]:
import geopandas as gpd
import pandas as pd

In [32]:
input_file = 'Census2011_Admin_Counties_generalised20m/Census2011_Admin_Counties_generalised20m.shp'
output_file = '../geo_files/Irish_counties.shp'

#Read in the shape file using GeoPandas
Geo = gpd.read_file(input_file)

In [33]:
#Remove ' County' from the end of each county name in column 'COUNTYNAME'. The only exceptions are
#counties with a city. These counties will have ' County' appended to their name. The county name on its
#own will refer to the total landmass contained in county borders, e.g. 'Galway City' refers to Galway city,
#'Galway' refers to the total landmass inside Galway's borders, while 'Galway County' refers to 'Galway'
#excluding 'Galway City'

#Remove ' County' from each name
Geo['COUNTYNAME'] = Geo['COUNTYNAME'].apply(lambda x : ''.join(x.split()[:-1]) if x.split()[-1] == 'County' else x)

#These are all of the cities according to the CSO
cities = ['Galway', 'Limerick', 'Waterford', 'Cork', 'Dublin']

#Append ' County' to these names to distinuish them from the city border and full county border
for city in cities:
    Geo.at[Geo[Geo['COUNTYNAME'] == city].index, 'COUNTYNAME'] = city + " County"

In [37]:
#This dictionary stores the regions in the form {'Region Name': ['List of Sub-Regions']}. The
#Regions stored in this dictionary will have their borders calculated using the borders of the Sub-Regions 
#and stored in the final output
aggregate = {'State': Geo['COUNTYNAME'].values,
             'Connacht': ['Galway County', 'Leitrim', 'Mayo','Roscommon', 'Sligo', 'Galway City'], 
             'Ulster': ['Cavan', 'Donegal','Monaghan'],
             'Munster': ['Clare', 'Cork County', 'Cork City', 'Kerry', 
                        'Limerick County', 'Limerick City', 'South Tipperary',  'North Tipperary', 
                         'Waterford County', 'Waterford City'],
             'Leinster': ['Carlow', 'Dublin City', 'South Dublin','Fingal',
                          'Dún Laoghaire-Rathdown','Kildare', 'Kilkenny', 
                         'Laois','Longford', 'Louth','Meath', 
                         'Offaly', 'Westmeath','Wexford', 'Wicklow'],
            'Dublin' : ['Dublin City', 'South Dublin','Fingal', 'Dún Laoghaire-Rathdown'],
            'Tipperary': ['North Tipperary', 'South Tipperary'],
            'Limerick': ['Limerick City', 'Limerick County'],
             'Galway': ['Galway City', 'Galway County'],
             'Cork': ['Cork City', 'Cork County'],
             'Waterford': ['Waterford City', 'Waterford County']
            } 

#final_df is the GeoDataFrame that will be written to disk
final_df = Geo[['COUNTYNAME','geometry']].copy()

#Loop over each region and calculate its border using the unary_union() method in GeoPandas.
#Then append this border to final_df. This new border is named after the region
for region in aggregate:
    
    #Create a temporary column called region, to filter out the correct areas
    final_df[region] = Geo['COUNTYNAME'].where(~Geo['COUNTYNAME'].isin(aggregate[region]), region)
    
    #Calculate the geometrt
    region_geometry = final_df[final_df[region] == region]['geometry'].unary_union
    
    
    #Append to the GeoDataFrame
    final_df = final_df.append({'COUNTYNAME': region, 'geometry': region_geometry}, ignore_index = True)
    
    
    #Drop a column that was temporarily created
    final_df.drop(region, inplace=True, axis=1)

In [38]:
#Sort the borders in alphabetical order and write to disk
final_df = final_df.sort_values(by = 'COUNTYNAME').reset_index(drop = True)
final_df.to_file(output_file)